Generating trials dataset using resampling procedure.

* Number of resampling iterations: 100 (based on bootstrapping stability analysis)
* Number of trials per resampling: N = 40 (based on data in monkey dataset so the number of trials is from experiments)

In [1]:
import csv
import pickle
import numpy as np

# import from scripts
import os
current_wd = os.getcwd()
os.chdir(os.path.abspath("..\\..\\..\\isttc\\scripts"))
#os.chdir(os.path.abspath("C:\\Users\\ipoch\\Documents\\repos\\isttc\\scripts"))
from cfg_global import project_folder_path
from spike_train_utils import get_trials, bin_trials
os.chdir(current_wd)

In [2]:
dataset_folder = project_folder_path + 'results\\synthetic\\dataset\\'

### Get data

In [4]:
with open(dataset_folder + 'spike_trains_tau100ms_alpha0_3_fr3_5hz_len600sec_1000.pkl','rb') as f:
    data = pickle.load(f)

all_spike_trains = data['spike_trains']
alphas = data['alphas']
fr_values = data['fr_values']
tau_ms = data['tau_ms']
duration_ms = data['duration_ms']

print(f'n spike trains {len(all_spike_trains)}, len {all_spike_trains[0][-1]/1000}, duration_ms {duration_ms}')

n spike trains 1000, len 599.4657698736828, duration_ms 600000


### Make trials 

#### Same n_trials and same trial len

In [ ]:
n_trials = 40  
m_iterations = 100

fs = 1000
bin_size = 50 # in ms
signal_len = int(10 * 60 * fs)
trial_len = int(1000 * (fs / 1000)) # 1000ms trials 

In [ ]:
trial_dict = {}
trial_binned_dict = {}

for i in range(len(all_spike_trains)):
    if (i % 100) == 0:
        print(f'Processing unit {i}')
    unit_trial_dict = {}
    unit_trial_binned_dict = {}
    for m in range(m_iterations):
        spikes = np.asarray([int(spike) for spike in all_spike_trains[i]])
        spikes_trials = get_trials(spikes, signal_len, n_trials, trial_len, verbose_=False)
        spikes_trials_binned = bin_trials(spikes_trials, trial_len, int(bin_size*(fs/1000)))
        unit_trial_dict[m] = spikes_trials
        unit_trial_binned_dict[m] = spikes_trials_binned

    trial_dict[i] = unit_trial_dict
    trial_binned_dict[i] = unit_trial_binned_dict

In [ ]:
with open(dataset_folder + 'trial_dict.pkl', 'wb') as f:
    pickle.dump(trial_dict, f)

with open(dataset_folder + 'trial_binned_dict.pkl', 'wb') as f:
    pickle.dump(trial_binned_dict, f)

#### Varying n_trails and trial len

In [11]:
m_iterations = 100
fs = 1000
bin_size = 50 # in ms
signal_len = int(10 * 60 * fs)

n_trials_min, n_trials_max = 10, 100  
trial_len_min, trial_len_max = 1000, 5*1000  

global_rng = np.random.default_rng(42)
n_trials_all = global_rng.integers(n_trials_min,
                              n_trials_max,
                              size=len(all_spike_trains),
                              endpoint=True)
possible_trial_len = np.arange(trial_len_min, trial_len_max+1, 500)
trial_lens_all = global_rng.choice(possible_trial_len, size=len(all_spike_trains))

In [12]:
trial_dict = {}
trial_binned_dict = {}

for i in range(len(all_spike_trains)):
    if (i % 100) == 0:
        print(f'Processing unit {i}')
    unit_trial_dict = {}
    unit_trial_binned_dict = {}

    n_trials = n_trials_all[i]
    trial_len = trial_lens_all[i]
    for m in range(m_iterations):
        spikes = np.asarray([int(spike) for spike in all_spike_trains[i]])
        spikes_trials = get_trials(spikes, signal_len, n_trials, trial_len, verbose_=False)
        spikes_trials_binned = bin_trials(spikes_trials, trial_len, int(bin_size*(fs/1000)))
        unit_trial_dict[m] = spikes_trials
        unit_trial_binned_dict[m] = spikes_trials_binned

    trial_dict[i] = unit_trial_dict
    trial_binned_dict[i] = unit_trial_binned_dict

Processing unit 0
Processing unit 100
Processing unit 200
Processing unit 300
Processing unit 400
Processing unit 500
Processing unit 600
Processing unit 700
Processing unit 800
Processing unit 900


In [16]:
trial_dict_full = {'trial_dict': trial_dict, 
                   'n_trials': n_trials_all, 
                   'trial_lens':trial_lens_all}
trial_binned_dict_full = {'trial_dict': trial_binned_dict, 
                   'n_trials': n_trials_all, 
                   'trial_lens':trial_lens_all}


with open(dataset_folder + 'trial_var_tau100ms_alpha0_3_fr3_5hz_len600sec_1000_dict.pkl', 'wb') as f:
    pickle.dump(trial_dict_full, f)

with open(dataset_folder + 'trial_binned_var_tau100ms_alpha0_3_fr3_5hz_len600sec_1000_dict.pkl', 'wb') as f:
    pickle.dump(trial_binned_dict_full, f)

In [19]:
n_trials_all

array([ 18,  80,  69,  49,  49,  88,  17,  73,  28,  18,  57,  98,  76,
        79,  75,  81,  56,  21,  86,  50,  55,  43,  26,  94,  81,  68,
        46,  84,  59,  50,  50,  30,  18,  60,  90,  15,  88,  85,  35,
        67,  25,  78,  73,  42,  16,  98,  50,  91,  71,  80,  79,  27,
        43,  52,  55,  13,  59,  24,  77,  72,  93,  77,  43,  98,  47,
        39,  92,  43,  16,  52,  82,  27,  52,  21,  72,  53,  40,  30,
        61,  70,  95,  49,  24,  85,  67,  73,  18,  38,  79,  85,  49,
        83,  86,  45,  91,  36,  31,  72,  67,  22,  85,  28,  83,  10,
        82,  81,  81,  70,  52,  74,  35,  81,  60,  51,  56,  61,  13,
        22,  32,  20,  50,  70,  69,  52,  87,  61,  17,  79,  62,  67,
        61,  60,  18,  60,  82,  37,  64,  12,  41,  49,  99,  29,  35,
        47, 100,  87,  13,  31,  84,  15,  87,  35,  93,  36,  49,  70,
        21,  60,  55,  81, 100,  70,  47,  46,  48,  84,  39,  25,  40,
        12,  19,  18,  80,  75,  73,  52,  75,  24,  91,  55,  9